In [ ]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
# Matrix preparation auxiliary functions

def prep_diags(N, d):
    # This performs the delta(x-x'+da) operation for the matrix
    if d == 0:
        return np.eye(N, dtype=complex)
    else:
        mat = np.zeros((N, N), dtype=complex)
        if d == 1:
            for i in range(N - 1):
                mat[i + 1][i] = 1
        else: # d == - 1
            for i in range(N - 1):
                mat[i][i + 1] = 1
        return mat

def mat2hext(mat, q):
    N = mat.shape[0]
    board = np.zeros((4 * N, 4 * N), dtype=complex)
    board[((q // 4) * N):((q // 4 + 1) * N), ((q % 4) * N):((q % 4 + 1) * N)] = mat
    return board

def dot_pauli(mat, pauli_idxs):
    # pauli_idxs should contain a two-char string, each in {1, x, y, z}
    id = np.eye(2, dtype=complex)
    sx = np.array([[0,1],[1,0]], dtype=complex)
    sy = np.array([[0,-1j],[1j,0]], dtype=complex)
    sz = np.array([[1,0],[0,-1]], dtype=complex)
    mat_dict = {'1': id, 'x': sx, 'y': sy, 'z': sz}

    return np.kron(np.kron(mat_dict[pauli_idxs[0]], mat_dict[pauli_idxs[1]]), mat.astype(complex))

def fourier_ham(N, b, M, ky, t):
    return b * np.sin(ky) * dot_pauli(prep_diags(N, 0), 'zx') + \
           b * (-1 / 2j) * dot_pauli(prep_diags(N, 1) - prep_diags(N, -1), 'zy') + \
           dot_pauli((M - 2 * t * np.cos(ky)) * prep_diags(N, 0) - t * (prep_diags(N, 1) + prep_diags(N, -1)), 'x1')

def expec(vec, mat):
    return np.real(vec.conj().T @ mat @ vec)

In [ ]:
# Predef
#USER: Change this whenever you want to test new values

M = 2
N = 20
b = 10

# Preliminary diagonalization

X_op = dot_pauli(np.diag(np.arange(-0.5*N+0.5, 0.5*N+0.5, 1)), '11')
SigzSx_op = dot_pauli(prep_diags(N, 0), 'zx')
Vy_op = lambda ky: b * np.cos(ky) * dot_pauli(prep_diags(N, 0), 'zx') + \
                   2 * dot_pauli(prep_diags(N, 0), 'x1')

ks = np.linspace(-np.pi, np.pi, N, endpoint=False)
eigvls = np.zeros((len(ks), 4 * N), dtype=complex)
eigvs = np.zeros((len(ks), 4 * N, 4 * N), dtype=complex)

for iy, ky in enumerate(ks):
    hamiltonian = fourier_ham(N, b, M, ky, 1)
    eigvls[iy, :], cur_eigvs = np.linalg.eigh(hamiltonian)
    eigvs[iy, :, :] = cur_eigvs

for band in range(4 * N):
    if band == 2 * N - 2 or band == 2 * N - 1: # Just verifying they are degenerate
        print(np.real(eigvls[:, band]))
    plt.plot(ks, eigvls[:, band])
plt.title(f"Band Structure for M={M}t, b={b}t")
plt.ylabel("Energy [t]")
plt.xlabel("ky [1/a]")
plt.show()

In [ ]:
ks_endpoints = np.linspace(-np.pi, np.pi, N + 1, endpoint=True)

eigvs_sorted = np.zeros((len(ks) + 1, 4 * N, 2), dtype=complex)
vy_expec = np.zeros((len(ks) + 1, 2))
sigzsx_expec = np.zeros((len(ks) + 1, 2))

for iy, ky in enumerate(ks):
    vedge1 = eigvs[iy, :, 2 * N - 2]
    vedge2 = eigvs[iy, :, 2 * N - 1]
    subspace = np.column_stack([vedge1, vedge2])

    X_proj = subspace.conj().T @ X_op @ subspace

    w, W = np.linalg.eigh(X_proj)
    phi_L, phi_R = subspace @ W[:,0], subspace @ W[:,1]
    phi_L /= np.linalg.norm(phi_L); phi_R /= np.linalg.norm(phi_R)

    # Log
    vy_expec[iy, :] = (expec(phi_L, Vy_op(ky)), expec(phi_R, Vy_op(ky)))
    sigzsx_expec[iy, :] = (expec(phi_L, SigzSx_op), expec(phi_R, SigzSx_op))
    eigvs_sorted[iy, :, 0] = phi_L
    eigvs_sorted[iy, :, 1] = phi_R

    print(f"""For ky={ky:.3f}:
    ⟨    vy   ⟩ | {vy_expec[iy, 0]:.3f} | {vy_expec[iy, 1]:.3f}
    ⟨sig_z s_x⟩ | {sigzsx_expec[iy, 0]:.3f} | {sigzsx_expec[iy, 1]:.3f}""")

eigvs_sorted[-1, :, :] = eigvs_sorted[0, :, :]
vy_expec[-1, :] = vy_expec[0, :]
sigzsx_expec[-1, :] = sigzsx_expec[0, :]

fig, axs = plt.subplots(3, 2)
fig.suptitle("Helicity Measurements for Edge States")
fig.tight_layout()
for i in range(2):
    axs[0, i].plot(ks_endpoints, vy_expec[:, i])
    axs[0, i].title.set_text(f"⟨vy⟩ - Band {i+1}")
    axs[1, i].plot(ks_endpoints, sigzsx_expec[:, i])
    axs[1, i].title.set_text(f"⟨sig_z s_x⟩ - Band {i+1}")
    axs[2, i].plot(ks_endpoints, vy_expec[:, i] * sigzsx_expec[:, i])
    axs[2, i].title.set_text(f"Helicity - Band {i+1}")

    for j in range(3):
        axs[j, i].set_xlabel("ky [1/a]")
        axs[j, i].set_ylabel("|uky|")

plt.show()

In [ ]:
X, Y = np.meshgrid(np.arange(0, N, 1), ks_endpoints)

for band in range(2):
    band += 0
    fig, axs = plt.subplots(2, 2, subplot_kw={"projection": "3d"})
    fig.suptitle(f"Eigenfunctions for M={M}t, Band {band+1}")

    surf = axs[0, 0].plot_surface(X, Y, np.abs(eigvs_sorted[:, :N, band]), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    axs[0, 0].title.set_text('Sublattice A, spin x up')

    surf = axs[0, 1].plot_surface(X, Y, np.abs(eigvs_sorted[:, N:2*N, band]), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    axs[0, 1].title.set_text('Sublattice A, spin x down')

    surf = axs[1, 0].plot_surface(X, Y, np.abs(eigvs_sorted[:, 2*N:3*N, band]), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    axs[1, 0].title.set_text(f"Sublattice B, spin x up")

    surf = axs[1, 1].plot_surface(X, Y, np.abs(eigvs_sorted[:, 3*N:, band]), cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    axs[1, 1].title.set_text(f"Sublattice B, spin x down")

    for i in range(4):
        axs[i // 2, i % 2].set_xlabel(f'x [a]')
        axs[i // 2, i % 2].set_ylabel(f"ky [1/a]")
        axs[i // 2, i % 2].set_zlabel(f"|uky(x)|")

    plt.show()